# evaluation.ipynb

**Projekt:** Spotify-Datensatz  
**Autor:** Erjon Hulaj  
**Datum:** 05.04.2025  

## Evaluation des Modells
In diesem Notebook analysiere ich die Qualität meines Vorhersagemodells anhand statistischer Metriken und einer Wahrheitsmatrix.

## (4.1) Aussagekräftige Felder

Für mein Modell zur Vorhersage der Popularität von Songs habe ich untersucht, welche Features besonders aussagekräftig sind.  
Durch Korrelationen, visuelle Analyse (Scatterplots) und Modelltraining haben sich folgende Felder als besonders wichtig herausgestellt:

- **energy**
- **danceability**
- **valence**
- **loudness**
- **duration_ms**

Diese Felder haben eine erkennbare Verbindung zur Popularität, da sie musikalische Eigenschaften beschreiben, die den Hörgenuss und damit die Beliebtheit beeinflussen können.


## (4.2) Messmetrik: MSE und R²

Ich habe den mittleren quadratischen Fehler (MSE) und das Bestimmtheitsmass (R²) verwendet, um mein Modell zu bewerten.  
- **MSE** zeigt, wie stark die Vorhersagen vom tatsächlichen Wert abweichen (je kleiner, desto besser).
- **R²** zeigt, wie viel der Zielvariable durch das Modell erklärt wird (max. 1.00).

Diese Metriken sind bei Regressionsmodellen Standard und sehr gut geeignet zur Bewertung der Modellgüte.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Datensatz laden
df = pd.read_csv("/Users/erjon/Library/CloudStorage/OneDrive-AlteKantonsschuleAarau/BBB/Module/2. Jahr/M259/LB/dataset.csv")

# Features (X) und Zielvariable (y)
X = df[["energy", "danceability", "loudness", "valence", "duration_ms"]]
y = df["popularity"]

# Aufteilen in Training und Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen
y_pred = model.predict(X_test)


In [3]:
from sklearn.metrics import mean_squared_error, r2_score

# Metriken berechnen
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mittlerer quadratischer Fehler (MSE):", round(mse, 2))
print("Bestimmtheitsmass (R²-Score):", round(r2, 2))


Mittlerer quadratischer Fehler (MSE): 488.86
Bestimmtheitsmass (R²-Score): 0.01


### Bewertung der Modellqualität mit Metriken

- **Mittlerer quadratischer Fehler (MSE)** liegt bei 488.86. Das bedeutet, dass die durchschnittliche quadratische Abweichung zwischen vorhergesagten und echten Popularitätswerten relativ hoch ist.
- **Bestimmtheitsmass (R²-Score)** beträgt 0.01. Das Modell erklärt also nur ca. 1 % der Varianz in den Popularitätswerten. Dies zeigt, dass das lineare Modell die Zusammenhänge in den Daten nur sehr schlecht abbildet.

→ Die Messmetriken bestätigen, dass das Modell noch deutliches Verbesserungspotenzial hat.


## (4.3) Wahrheitsmatrix, Recall & Precision

Zur Bewertung meines Modells als Klassifikation habe ich die Popularität in zwei Klassen eingeteilt:  
- Beliebt (>= 50)  
- Nicht beliebt (< 50)

Mit Hilfe einer Confusion Matrix konnte ich **Precision** (Genauigkeit) und **Recall** (Trefferquote) berechnen.  
Diese zeigen, wie zuverlässig das Modell beliebte Songs erkennt. Dies ist besonders hilfreich, wenn man etwa gezielt nur erfolgreiche Songs empfehlen möchte.


In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Popularity binär einstufen: beliebt (>=50) = 1, sonst 0
y_true_class = (y_test >= 50).astype(int)
y_pred_class = (y_pred >= 50).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_true_class, y_pred_class)
print("Wahrheitsmatrix (Confusion Matrix):")
print(cm)

# Recall, Precision usw.
print("\nAuswertung:")
print(classification_report(y_true_class, y_pred_class))


Wahrheitsmatrix (Confusion Matrix):
[[17005     0]
 [ 5795     0]]

Auswertung:
              precision    recall  f1-score   support

           0       0.75      1.00      0.85     17005
           1       0.00      0.00      0.00      5795

    accuracy                           0.75     22800
   macro avg       0.37      0.50      0.43     22800
weighted avg       0.56      0.75      0.64     22800



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### Bewertung der Klassifikation

Die Confusion Matrix zeigt, dass das Modell alle Songs als „nicht beliebt“ einstuft. Dadurch erkennt es keine tatsächlich beliebten Songs (Recall = 0.00), und es gibt auch keine echten positiven Vorhersagen (Precision = 0.00).

Das Modell ist somit nicht fähig, beliebte Songs zuverlässig zu identifizieren. Es tendiert stark dazu, alles als „nicht beliebt“ zu bewerten. Sehr wahrscheinlich ist das so, weil die Popularitätswerte im Trainingssatz mehrheitlich unter der Schwelle von 50 liegen. Für diese Art der Klassifikation ist das Modell ungeeignet und müsste durch komplexere Methoden ersetzt werden.


### (4.4) Fazit zur Modellqualität

Mein Modell kann grobe Trends in den Daten erkennen, liefert jedoch keine präzisen Vorhersagen.  
Der **MSE** ist hoch und das **R²** liegt nahe bei 0.  
Auch die Klassifikation zeigt, dass das Modell beliebte Songs nicht zuverlässig erkennt (**Recall** und **Precision** = 0 für Klasse 1).  

→ **Hypothese:** Vermutlich reichen die gewählten Features nicht aus, um die Komplexität der Popularität vollständig abzubilden.  
Mögliche Verbesserungen wären nicht-lineare Modelle (z. B. Entscheidungsbäume) oder weitere Features wie Genres, Interpreten oder Veröffentlichung.